# Spacy 3.0 Tutorial

We use spacy 3.0 for natural language processing

## Install French Dependencies

In [1]:
import spacy
!python -m spacy download fr_core_news_sm

You should consider upgrading via the '/home/dynamo63/Documents/python-projects/streamlit-nltk/env/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


## Get Started

We use spacy

In [2]:
import fr_core_news_sm

nlp = fr_core_news_sm.load()


Text the model

In [3]:
doc = nlp("Python est le meilleur language de programmation. Javascript est aussi un bon choix.")
for ent in doc.ents:
  print(ent.text + " : " + ent.label_)

Python : PER
Javascript : MISC


## Create Entity with spacy

In [4]:
nlp.pipe_names

['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

Create data for spacy.


1.   We create a function for create data



In [6]:
import re
def create_data_entities(sentence: str, keyword: str, categorie: str) -> tuple:
  """
    We create a row data for spacy entities
  """
  match = re.search(keyword, sentence)
  if match:
    return sentence, {"entities": [(match.start(), match.end(), categorie)]}
  else:
    return None


In [7]:
text_1 = "Charlie, Ela, Marius et Cicero sont ânes qui ne demandent pas mieux que de vous accompagner en balade"
text_2 = "La tour d'Autriche marque la fin de votre visite et vous offre une magnifique vue à sur la ville de Bouillon"
text_3 = "cette belle promenade le conduira dans la vallée de la Trouille et autour du Mont Panisel par de petits sentiers escarpés"
td_1 = create_data_entities(text_1, 'balade', 'BALADE')
td_2 = create_data_entities(text_2, 'visite', 'BALADE')
td_3 = create_data_entities(text_3, 'promenade', 'BALADE')
train_data = [td_1, td_2, td_3,
              create_data_entities("Nouvelle activité pour enfants alliant promenade et découverte grâce aux jeux et défis", 'promenade', 'BALADE'),
              create_data_entities("Une visite du Fort de Lantin", 'visite', 'BALADE'),
              create_data_entities("Parcours audio guidé sur smartphone: Le Petit Patrimoine au centre d’Olne", 'Parcours', 'BALADE'),
              create_data_entities("C'est une belle promenade au bord de mer", 'promenade', 'BALADE'),
              create_data_entities("Voulez-vous explorer la belle Kempen d'une manière unique", 'explorer', 'BALADE'),
              create_data_entities("Excursion d'une journée dans votre propre pays", 'Excursion', 'BALADE'),
              create_data_entities("Visitez notre hall d'expositions", 'Visitez', 'BALADE'),
              ]
train_data


[('Charlie, Ela, Marius et Cicero sont ânes qui ne demandent pas mieux que de vous accompagner en balade',
  {'entities': [(95, 101, 'BALADE')]}),
 ("La tour d'Autriche marque la fin de votre visite et vous offre une magnifique vue à sur la ville de Bouillon",
  {'entities': [(42, 48, 'BALADE')]}),
 ('cette belle promenade le conduira dans la vallée de la Trouille et autour du Mont Panisel par de petits sentiers escarpés',
  {'entities': [(12, 21, 'BALADE')]}),
 ('Nouvelle activité pour enfants alliant promenade et découverte grâce aux jeux et défis',
  {'entities': [(39, 48, 'BALADE')]}),
 ('Une visite du Fort de Lantin', {'entities': [(4, 10, 'BALADE')]}),
 ('Parcours audio guidé sur smartphone: Le Petit Patrimoine au centre d’Olne',
  {'entities': [(0, 8, 'BALADE')]}),
 ("C'est une belle promenade au bord de mer",
  {'entities': [(16, 25, 'BALADE')]}),
 ("Voulez-vous explorer la belle Kempen d'une manière unique",
  {'entities': [(12, 20, 'BALADE')]}),
 ("Excursion d'une journée dan

In [10]:
# get pipe
ner = nlp.get_pipe('ner')
LABEL = 'BALADE'
# Add the new label to ner
ner.add_label(LABEL)
# resume training for create optimizer
optimizer = nlp.resume_training()
move_names = list(ner.move_names)

# List of pipes you want to train
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]

# List of pipes which should remain unaffected in training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [16]:
# Importing requirements
from spacy.util import minibatch, compounding
from spacy.training.example import Example
import random

# Begin training by disabling other pipeline components
with nlp.disable_pipes(*other_pipes) :
  sizes = compounding(1.0, 4.0, 1.001)
  for i in range(10):
    # shuffle examples before training
    random.shuffle(train_data)
    # batch up the examples using spaCy's minibatch
    batches = minibatch(train_data, size=sizes)
    # get loss
    loss = {}
    for batch in batches:
      for text, annotations in batch:
        # create Example
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        # Update the model
        nlp.update([example], losses=loss, drop=0.3)
        print(loss)


{'ner': 5.079400234577947}
{'ner': 7.97815791782277}
{'ner': 11.748896244630304}
{'ner': 21.058202289061594}
{'ner': 23.057992015162192}
{'ner': 28.05236956610367}
{'ner': 29.81155029280037}
{'ner': 32.23190352971501}
{'ner': 34.1680726906374}
{'ner': 38.82559926896518}
{'ner': 2.9792833990484553}
{'ner': 6.251909427809466}
{'ner': 12.033810299143084}
{'ner': 13.906444465090232}
{'ner': 15.966311951193262}
{'ner': 17.935746918363208}
{'ner': 19.79884194044635}
{'ner': 21.65897268613108}
{'ner': 23.44020529787295}
{'ner': 25.636728863031664}
{'ner': 1.6397199765779078}
{'ner': 3.736125007388182}
{'ner': 5.255185907240957}
{'ner': 6.946792587754317}
{'ner': 8.59236802731175}
{'ner': 10.547089706291445}
{'ner': 12.523377914098091}
{'ner': 13.816187335643917}
{'ner': 15.864808406797238}
{'ner': 17.510975238285027}
{'ner': 1.5812010171357542}
{'ner': 3.060192119097337}
{'ner': 4.4252897694241256}
{'ner': 5.933664583862992}
{'ner': 7.5472373960510595}
{'ner': 10.118975229052012}
{'ner': 11.6

In [17]:
from spacy import displacy
doc = nlp("Visitez notre hall d'expositions")
displacy.render(doc, style="ent", jupyter=True)

Save the model

In [18]:
nlp.to_disk('./spacy_ner_custom')